In [1]:
import mwparserfromhell
import pywikibot
import wikipedia
from wikipedia import WikipediaPage
from bs4 import BeautifulSoup as bs
import requests

In [2]:
# Some Note in Advance
# Using wikipedia for finding synonyms and paraphrases of Named Entities would
# include using the redirections for each page for perfect results
# This can be added
# The keyword "Myanmar" will not return "Burma" ATM as Burma is a redirection page

In [3]:
# This would be the keyword search
keyword = "Elizabeth II"

In [4]:
result = wikipedia.search(keyword)

result = result[0]

In [5]:
page = WikipediaPage(keyword)
keyword_url = page.url

In [8]:
API_URL = "https://en.wikipedia.org/w/api.php"

def parse(title):
    params = {
        "action": "query",
        "prop": "revisions",
        "rvprop": "content",
        "rvslots": "main",
        "rvlimit": 1,
        "titles": title,
        "format": "json",
        "formatversion": "2",
    }
    headers = {"User-Agent": "Me"}
    req = requests.get(API_URL, headers=headers, params=params)
    res = req.json()
    revision = res["query"]["pages"][0]["revisions"][0]
    text = revision["slots"]["main"]["content"]
    return mwparserfromhell.parse(text)


links = parse(keyword).filter_wikilinks()

In [7]:
results = {}

# For Elisabeth, there are >700 articles in "links"
# Using not the whole list of links would also be an option

for link in links:
    try:
        # preprocess the link
        wiki = WikipediaPage(str(link)[2:(len(str(link))-2)])
        keyword_url = wiki.url
        res = requests.get(keyword_url)
        soup = bs(res.text, "html.parser")
        for soup in soup.find_all("a"):
            url = soup.get("href", "")
            # Idea for stating the conditon later: Find wiki link to keyword
            if url.endswith("wiki/"+keyword.replace(" ", "_")):
                results[soup.text.strip()] = url
    except:
        pass


results

{'Elizabeth II': '/wiki/Elizabeth_II',
 'Queen Elizabeth II': '/wiki/Elizabeth_II',
 'Elizabeth\xa0II': '/wiki/Elizabeth_II',
 'her': '/wiki/Elizabeth_II',
 'Elizabeth': '/wiki/Elizabeth_II',
 'Princess Elizabeth': '/wiki/Elizabeth_II',
 'The Queen': '/wiki/Elizabeth_II',
 'Elizabeth II of the United Kingdom': '/wiki/Elizabeth_II',
 'the Queen': '/wiki/Elizabeth_II',
 'Princess (later Queen) Elizabeth': '/wiki/Elizabeth_II',
 'Queen': '/wiki/Elizabeth_II',
 'Queen Elizabeth II of the United Kingdom': '/wiki/Elizabeth_II',
 'Lilibet': '/wiki/Elizabeth_II',
 'Princess\xa0Elizabeth, Duchess\xa0of\xa0Edinburgh': '/wiki/Elizabeth_II',
 'Princess Elizabeth, Duchess of Edinburgh': '/wiki/Elizabeth_II',
 'Her Majesty the Queen': '/wiki/Elizabeth_II',
 'Queen Elizabeth': '/wiki/Elizabeth_II',
 'monarch': '/wiki/Elizabeth_II',
 'HM Queen Elizabeth II': '/wiki/Elizabeth_II',
 'HM The Queen': '/wiki/Elizabeth_II',
 'Princess Elizabeth of the United Kingdom': '/wiki/Elizabeth_II',
 'QueenElizabeth 

In [ ]:
# Additional step:
# now delete any word from the list that is not a named entity (like "her"/ "monarch")
# Also some basic preprocessing (for "Princess\xa0Elizabeth" etc.)